# CLR analysis

In [1]:

# load libraries
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(tidyverse)
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
#quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
#quiet_library(Nebulosa)
quiet_library(future)
quiet_library(future.apply)
#quiet_library(H5weaver)
quiet_library(SingleCellExperiment)
#quiet_library(scCustomize)
quiet_library(SeuratDisk)
#quiet_library(SeuratData)
quiet_library(ggpubr)
quiet_library(knitr)
quiet_library(plotly)
quiet_library(lme4)
quiet_library(ggpubr)

In [2]:
# Check number of cores
future::availableCores()
# Set up parallel processing to run when using 'future' functions 
future::plan(strategy = "multicore", workers = future::availableCores()-5)  
options(future.globals.maxSize = 1000 * 1024^3)
# to turn off parallel processing run line below
# future::plan(strategy = "sequential")

system 
    64

In [3]:
# define the color palette to be used
npg_color <- c("#E64B35FF", "#4DBBD5FF", "#00A087FF", "#3C5488FF", "#F39B7FFF", 
               "#8491B4FF", "#91D1C2FF", "#DC0000FF", "#7E6148FF", "#B09C85FF")
nejm_color <- c("#BC3C29FF", "#0072B5FF", "#E18727FF", "#20854EFF", "#7876B1FF", "#6F99ADFF", "#FFDC91FF", "#EE4C97FF")
jama_color <- c("#374E55FF", "#DF8F44FF", "#00A1D5FF", "#B24745FF", "#79AF97FF", "#6A6599FF", "#80796BFF")
jco_color <- c("#0073C2FF", "#EFC000FF", "#868686FF", "#CD534CFF", "#7AA6DCFF", "#003C67FF", "#8F7700FF")
cluster_colors <- c("#DC050C", "#FB8072", "#1965B0", "#7BAFDE", "#882E72", "#B17BA6", "#FF7F00", "#FDB462", "#E7298A", 
    "#E78AC3", "#33A02C", "#B2DF8A", "#55A1B1", "#8DD3C7", "#A6761D", "#E6AB02", "#7570B3", "#BEAED4", "#666666", "#999999", 
    "#aa8282", "#d4b7b7", "#8600bf", "#ba5ce3", "#808000", "#aeae5c", "#1e90ff", "#00bfff", "#56ff0d", "#ffff00")

cluster_colors_ext <- colorRampPalette(cluster_colors)(36)
options(repr.plot.width = 10, repr.plot.height = 10)

# Helper functions

In [4]:
# Defnie a function to do clr transformatin from frequency table
FreqClrTran <- function(freq_table, freq_col='frequency_live', celltype_col='labels', sample_col='sample_id'){
    require('compositions')
    freq <- freq_table %>% select(all_of(c(freq_col, celltype_col, sample_col))) %>% 
        pivot_wider(id_cols = .data[[sample_col]], names_from =.data[[celltype_col]],
                    values_from = .data[[freq_col]], values_fill=0)
    freq_mx <- freq %>% select(-.data[[sample_col]]) %>% as.matrix()
    rownames(freq_mx) <- freq[[sample_col]]
    freq_clr <- compositions::clr(freq_mx) %>% as_tibble(rownames = sample_col) %>% 
        pivot_longer(cols=c(-.data[[sample_col]]), names_to = celltype_col, values_to = 'clr') 
    freq_meta_clr <- freq_table %>% full_join(freq_clr, by = c(sample_col,celltype_col))
    return(freq_meta_clr)
}


In [23]:
# read the frequency file
Freq_file=read.csv("/home/jupyter/ALTRA_ANALYSIS/Deep_clean/certpro/output_files/CD4T_mem_certpro_Cluster_freq_res_0.4_py.csv")
Freq_file%>%head(3)

,X,X0,X1,X2,X3,X4,X5,X6,X7,X8,⋯,X10,X11,X12,X13,X14,X15,X16,X17,X18,sample.sampleKitGuid
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,KT00052,0.1042383,0.07073310,0.09621993,0.06872852,0.07646048,0.09765178,0.07273769,0.05126002,0.06386025,⋯,0.06386025,0.02978236,0.04696449,0.005727377,0.02949599,0.02691867,0.02290951,0.016609393,0.002290951,KT00052
2,KT00055,0.1823836,0.02367576,0.05136437,0.08667737,0.03972713,0.06059390,0.09731140,0.05457464,0.03390851,⋯,0.04313804,0.02929374,0.04674960,0.110152488,0.04373997,0.01444623,0.02407705,0.008226324,0.000000000,KT00055
3,KT00056,0.1757411,0.04779189,0.12371446,0.06352087,0.08197217,0.06563823,0.07743497,0.03690260,0.05293406,⋯,0.06261343,0.02631579,0.02964307,0.002722323,0.03781004,0.02903811,0.01482154,0.007864489,0.000000000,KT00056


In [24]:
Freq_file1=Freq_file%>%select(-X)
Freq_file1%>%head(3)

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,sample.sampleKitGuid
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.1042383,0.07073310,0.09621993,0.06872852,0.07646048,0.09765178,0.07273769,0.05126002,0.06386025,0.05355097,0.06386025,0.02978236,0.04696449,0.005727377,0.02949599,0.02691867,0.02290951,0.016609393,0.002290951,KT00052
2,0.1823836,0.02367576,0.05136437,0.08667737,0.03972713,0.06059390,0.09731140,0.05457464,0.03390851,0.04995987,0.04313804,0.02929374,0.04674960,0.110152488,0.04373997,0.01444623,0.02407705,0.008226324,0.000000000,KT00055
3,0.1757411,0.04779189,0.12371446,0.06352087,0.08197217,0.06563823,0.07743497,0.03690260,0.05293406,0.06352087,0.06261343,0.02631579,0.02964307,0.002722323,0.03781004,0.02903811,0.01482154,0.007864489,0.000000000,KT00056


In [25]:
# Replace column names starting with "X" with "cluster"
new_colnames <- gsub("^X", "cluster", colnames(Freq_file1))

# Assign new column names to the data frame
colnames(Freq_file1) <- new_colnames

In [26]:
Freq_file1%>%head(3)

,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11,cluster12,cluster13,cluster14,cluster15,cluster16,cluster17,cluster18,sample.sampleKitGuid
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.1042383,0.07073310,0.09621993,0.06872852,0.07646048,0.09765178,0.07273769,0.05126002,0.06386025,0.05355097,0.06386025,0.02978236,0.04696449,0.005727377,0.02949599,0.02691867,0.02290951,0.016609393,0.002290951,KT00052
2,0.1823836,0.02367576,0.05136437,0.08667737,0.03972713,0.06059390,0.09731140,0.05457464,0.03390851,0.04995987,0.04313804,0.02929374,0.04674960,0.110152488,0.04373997,0.01444623,0.02407705,0.008226324,0.000000000,KT00055
3,0.1757411,0.04779189,0.12371446,0.06352087,0.08197217,0.06563823,0.07743497,0.03690260,0.05293406,0.06352087,0.06261343,0.02631579,0.02964307,0.002722323,0.03781004,0.02903811,0.01482154,0.007864489,0.000000000,KT00056


In [28]:

value_vars <- c("cluster0", "cluster1", "cluster2", "cluster3", "cluster4", 
                   "cluster5", "cluster6", "cluster7", "cluster8", "cluster9",
                   "cluster10", "cluster11", "cluster12", "cluster13", 
                   "cluster14", "cluster15", "cluster16", "cluster17", 
                   "cluster18"#, "cluster19","cluster20"
                   )
# Pivot the DataFrame long with 'clusters' as the new column
pivoted_df <- Freq_file1 %>%
  pivot_longer(cols = all_of(value_vars), names_to = 'clusters', values_to = 'frequency_live')

# Display the pivoted DataFrame
head(pivoted_df)

sample.sampleKitGuid,clusters,frequency_live
<chr>,<chr>,<dbl>
KT00052,cluster0,0.10423826
KT00052,cluster1,0.07073310
KT00052,cluster2,0.09621993
KT00052,cluster3,0.06872852
KT00052,cluster4,0.07646048
KT00052,cluster5,0.09765178


In [29]:
pivoted_df1_clr <- FreqClrTran(pivoted_df, freq_col='frequency_live',
                                   celltype_col='clusters', 
                          sample_col='sample.sampleKitGuid')

In [30]:
pivoted_df1_clr%>%head()
pivoted_df1_clr%>%dim

sample.sampleKitGuid,clusters,frequency_live,clr
<chr>,<chr>,<dbl>,<dbl>
KT00052,cluster0,0.10423826,0.9848633
KT00052,cluster1,0.07073310,0.5970978
KT00052,cluster2,0.09621993,0.9048206
KT00052,cluster3,0.06872852,0.5683484
KT00052,cluster4,0.07646048,0.6749581
KT00052,cluster5,0.09765178,0.9195919


[1] 2679    4

In [ ]:
# read the ALTRA clinical Meta Data to add the required columns to the frequency table 

In [31]:
meta=read.csv("/home/jupyter/ALTRA_ANALYSIS/Deep_clean/AIM3/input_csv_files/2023-11-22_ALTRA_Metadata_labs.csv")

In [32]:
filt_meta_ot <- select(meta,BMI, days_to_conversion, ageAtDraw,sample.sampleKitGuid, subject.biologicalSex, subject.subjectGuid,bmi_conv,age_conv,Status_Long,Status_Xsec,Age2023,anti_ccp3,anti_ccp3_finalCombined,
                    anti_ccp31  )

In [33]:
filt_meta_ot%>%dim()
filt_meta_ot%>%head(3)

[1] 157  14

,BMI,days_to_conversion,ageAtDraw,sample.sampleKitGuid,subject.biologicalSex,subject.subjectGuid,bmi_conv,age_conv,Status_Long,Status_Xsec,Age2023,anti_ccp3,anti_ccp3_finalCombined,anti_ccp31
,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,24.65716,-714,56,KT00052,Female,CU1009,25.64892,58,pre,at_risk,60,88,88,74
2,43.24286,NA,53,KT00055,Female,CU1008,NA,NA,NA,early_RA,57,23,23,10
3,25.94834,-632,42,KT00056,Female,CU1007,26.19498,44,pre,at_risk,46,262,2544,262


In [34]:
intersection%>%length

[1] 141

In [35]:
# Find the intersection of sample.sampleKitGuid values
intersection <- intersect(filt_meta_ot$sample.sampleKitGuid, pivoted_df1_clr$sample.sampleKitGuid)

# Subset filt_meta and merged_a3_th17 based on the intersection
filt_meta_ot_intersect <- filt_meta_ot[filt_meta_ot$sample.sampleKitGuid %in% intersection, ]
pivoted_df1_clr_intersect <- pivoted_df1_clr[pivoted_df1_clr$sample.sampleKitGuid %in% intersection, ]

# Perform the merge based on the intersection
merged_filt_ot<- merge(filt_meta_ot_intersect, pivoted_df1_clr_intersect, by = 'sample.sampleKitGuid', all.x = TRUE)

In [36]:
merged_filt_ot%>%dim()
merged_filt_ot%>%head(3)

[1] 2679   17

,sample.sampleKitGuid,BMI,days_to_conversion,ageAtDraw,subject.biologicalSex,subject.subjectGuid,bmi_conv,age_conv,Status_Long,Status_Xsec,Age2023,anti_ccp3,anti_ccp3_finalCombined,anti_ccp31,clusters,frequency_live,clr
,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>
1,KT00052,24.65716,-714,56,Female,CU1009,25.64892,58,pre,at_risk,60,88,88,74,cluster0,0.10423826,0.9848633
2,KT00052,24.65716,-714,56,Female,CU1009,25.64892,58,pre,at_risk,60,88,88,74,cluster1,0.07073310,0.5970978
3,KT00052,24.65716,-714,56,Female,CU1009,25.64892,58,pre,at_risk,60,88,88,74,cluster2,0.09621993,0.9048206


In [37]:
merged_filt_ot%>%
write.csv("/home/jupyter/ALTRA_ANALYSIS/Deep_clean/certpro/output_files/CD4T_mem_certpro_Frq_clr_res_0_4(c0-c18)_R.csv")